## Preparación de los dataframes para la base de datos

In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv(f"../datos/df_supermercados.csv")

Tabla supermercados

In [14]:
df_super = pd.DataFrame(df["supermercado"].unique())
df_super.reset_index(inplace=True)
df_super.rename(columns={'index':'id_super', 0:'nombre'}, inplace=True)
df_super["id_super"]=df_super["id_super"].apply(lambda x: x+1)

In [15]:
df_super.to_csv("../datos/tabla_super.csv")

In [16]:
df_super

,id_super,nombre
0,1,alcampo
1,2,carrefour
2,3,dia
3,4,eroski
4,5,hipercor
5,6,mercadona


Tabla productos

In [17]:
df_productos = pd.DataFrame(df["producto"].unique())
df_productos.reset_index(inplace=True)
df_productos.rename(columns={'index':'id_producto', 0:'nombre'}, inplace=True)
df_productos["id_producto"]=df_productos["id_producto"].apply(lambda x: x+1)

In [18]:
df_productos.to_csv("../datos/tabla_productos.csv")

In [19]:
df_productos

,id_producto,nombre
0,1,aceite de girasol
1,2,aceite de oliva
2,3,leche


Tabla comparativa

Incluimos los id del supermercado y producto para tener las claves foraneas:

In [20]:
df_merged = df.merge(right=df_super, how='left', left_on ='supermercado', right_on="nombre")
df_merged=df_merged.drop(columns=["Unnamed: 0", "supermercado", "nombre"])

In [21]:
df_merged_2 = df_merged.merge(right=df_productos, how='left', left_on ='producto', right_on="nombre")
df_merged_2 = df_merged_2.drop(columns=["producto", "nombre"])

Transformamos el precio a float:

In [22]:
df_merged_2["Precio (€)"]=df_merged_2["Precio (€)"].str.replace(",",".").apply(float)
df_merged_2["porcentaje"]=df_merged_2["porcentaje"].str.replace(",",".").apply(float)

Transformamos la fecha a formato fecha:

In [23]:
df_merged_2["Día"]=pd.to_datetime(df_merged_2["Día"])

Reordenamos y borramos duplicados:

In [24]:
df_merged_2 = df_merged_2.reindex(columns=['id_super', 'id_producto', 'nombre_producto', 'subcategoria', 'Día', 'Precio (€)', 'incremento', 'porcentaje'])
df_merged_2.drop_duplicates(inplace=True)

In [25]:
df_merged_2.to_csv("../datos/tabla_comparativa.csv")

In [26]:
df_merged_2.sample(5)

,id_super,id_producto,nombre_producto,subcategoria,Día,Precio (€),incremento,porcentaje
133313,6,3,leche semidesnatada sin lactosa hacendado 9 l,semidesnatada,2024-09-20,9.18,NaN,NaN
69277,4,2,aceite de oliva virgen extra arronizarbe nekea...,virgen extra,2024-09-26,8.38,NaN,NaN
25916,1,3,pascual leche de vaca desnatada de bienestar a...,desnatada,2024-09-30,10.08,NaN,NaN
110760,5,3,asturiana leche desnatada con calcio y vitamin...,desnatada,2024-08-15,1.48,NaN,NaN
41935,2,2,aceite de oliva virgen extra retama garrafa 5 l,virgen extra,2024-08-06,48.90,NaN,NaN
